In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.mam.*
import jline.util.matrix.*
import kotlin.math.*

In [ ]:
val model = Network("model")

val M = 4
// Create 4 single-server queues with PS scheduling
val nodes = Array<Node?>(4) { null }
for (i in 0 until M) {
    nodes[i] = Queue(model, "Queue${i+1}", SchedStrategy.PS)
    // Single server by default (numberOfServers = 1)
}

val source = Source(model, "Source")
val sink = Sink(model, "Sink")

// Create job classes
val closedClass = ClosedClass(model, "ClosedClass", 100, nodes[0], 0)  // High population
val openClass = OpenClass(model, "OpenClass", 0)

// Set service rates
for (i in 0 until M) {
    nodes[i]!!.setService(closedClass, Exp((i + 1).toDouble()))
    nodes[i]!!.setService(openClass, Exp(sqrt((i + 1).toDouble())))
}

// Set arrival rate for open class
source.setArrival(openClass, Exp(0.3))

println("Created 4 single-server PS queues with service rates 1, 2, 3, 4")
println("Closed class: 100 jobs, Open class arrivals: Exponential(0.3)")

In [ ]:
// Setup routing matrix
val numStations = model.numberOfStations
val numClasses = model.numberOfClasses

val P = model.initRoutingMatrix()

// Closed class routing: Queue1 -> Queue2 -> Queue3 -> Queue4 -> Queue1 (circular)
P.set(closedClass, closedClass, nodes[0], nodes[1], 1.0)
P.set(closedClass, closedClass, nodes[1], nodes[2], 1.0)
P.set(closedClass, closedClass, nodes[2], nodes[3], 1.0)
P.set(closedClass, closedClass, nodes[3], nodes[0], 1.0)

// Open class routing: Source -> Queue1 -> Queue2 -> Queue3 -> Sink (excludes Queue4)
P.set(openClass, openClass, source, nodes[0], 1.0)
P.set(openClass, openClass, nodes[0], nodes[1], 1.0)
P.set(openClass, openClass, nodes[1], nodes[2], 1.0)
P.set(openClass, openClass, nodes[2], sink, 1.0)

model.link(P)

println("\nRouting configured:")
println("  Closed class: Queue1->Queue2->Queue3->Queue4->Queue1 (circular)")
println("  Open class: Source->Queue1->Queue2->Queue3->Sink (excludes Queue4)")

In [ ]:
println("\nThis example shows the execution of the solver on a 2-class mixed model with 4 single server nodes.")

// Create solvers with appropriate configurations
// Note: CTMC is infinite on this model due to high population
val solvers = mutableListOf<NetworkSolver>()

// Default options matching MATLAB
val seed = 23000
val samples = 100000  // 1e5 in MATLAB
val cutoff = 3
val verbose = true
val keep = false

try {
    solvers.add(JMT(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed, "samples", samples))
} catch (e: Exception) {
    println("JMT solver not available: ${e.message}")
}

try {
    // SSA with infinite cutoff as in MATLAB (optionssa.cutoff = Inf)
    solvers.add(SSA(model, "cutoff", Double.POSITIVE_INFINITY, "seed", seed, "verbose", false, "samples", samples))
} catch (e: Exception) {
    println("SSA solver not available: ${e.message}")
}

try {
    solvers.add(FLD(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))
} catch (e: Exception) {
    println("Fluid solver not available: ${e.message}")
}

try {
    solvers.add(MVA(model, "method", "lin"))
} catch (e: Exception) {
    println("MVA solver not available: ${e.message}")
}

try {
    solvers.add(NC(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))
} catch (e: Exception) {
    println("NC solver not available: ${e.message}")
}

try {
    solvers.add(MAM(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))
} catch (e: Exception) {
    println("MAM solver not available: ${e.message}")
}

println("\nAvailable solvers: ${solvers.size}")
println("Note: CTMC is omitted due to infinite state space with high population")

In [ ]:
// Solve and display resultssolvers.forEachIndexed { index, solver ->    try {        println("\n${"+".repeat(50)}")                println("${"+".repeat(50)}")                val avgTable = solver.avgTable        avgTable.print()            } catch (e: Exception) {        println("\nSOLVER ${solver.name.replace("Solver", "")} FAILED: ${e.message}")    }}

## Results Analysis

This high-congestion mixed queueing network with PS scheduling demonstrates:

1. **Processor Sharing Benefits**: Fair resource allocation under heavy load
2. **High Population Handling**: PS generally handles congestion better than FCFS
3. **Solver Diversity**: Multiple numerical methods for comprehensive analysis
4. **Mixed Class Interactions**: How closed and open classes compete for resources

**PS Advantages over FCFS:**
- **Fairness**: All jobs receive equal share of service capacity
- **Predictability**: Response times less sensitive to job ordering
- **Stability**: Better performance under variable load conditions

**Expected Results:**
- **Lower Variance**: Response times should be more predictable than FCFS
- **Resource Sharing**: Open and closed classes share capacity more fairly
- **Bottleneck Effects**: Queue1 still likely bottleneck but with smoother performance

**Solver Performance:**
- **JMT/SSA**: Simulation methods provide ground truth
- **Fluid**: Continuous approximation for large populations
- **MVA/NC/MAM**: Analytical/numerical methods with approximations